In [1]:
!pip install streamlit pyngrok plotly --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.5 MB/s eta 0:00:00


In [10]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="FitPulse Dashboard", layout="wide")

st.title("🏃 FitPulse Health Anomaly Dashboard")

uploaded_file = st.file_uploader("Upload Fitness Data (CSV / JSON)", type=["csv","json"])

if uploaded_file:
    if uploaded_file.name.endswith(".csv"):
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_json(uploaded_file)

    df['date'] = pd.to_datetime(df['date'])

    st.subheader("Dataset Preview")
    st.dataframe(df.head())


    df['hr_anomaly'] = abs(
        (df['avg_heart_rate'] - df['avg_heart_rate'].mean())
        / df['avg_heart_rate'].std()
    ) > 2

    df['sleep_anomaly'] = df['total_sleep_minutes'] < 300
    df['steps_anomaly'] = df['TotalSteps'] < 2000

    start_date = st.date_input("Start Date", df['date'].min())
    end_date = st.date_input("End Date", df['date'].max())

    df = df[(df['date'] >= pd.to_datetime(start_date)) &
            (df['date'] <= pd.to_datetime(end_date))]


    st.subheader("Heart Rate Trend")
    fig_hr = px.line(df, x='date', y='avg_heart_rate')
    fig_hr.add_scatter(
        x=df[df['hr_anomaly']]['date'],
        y=df[df['hr_anomaly']]['avg_heart_rate'],
        mode='markers',
        marker=dict(color='red', size=8),
        name='Anomaly'
    )
    st.plotly_chart(fig_hr, use_container_width=True)


    st.subheader(" Sleep Duration")
    fig_sleep = px.line(df, x='date', y='total_sleep_minutes')
    st.plotly_chart(fig_sleep, use_container_width=True)

    st.subheader("👣 Step Count")
    fig_steps = px.bar(df, x='date', y='TotalSteps')
    st.plotly_chart(fig_steps, use_container_width=True)


    st.subheader(" Download Anomaly Report")
    report = df[(df['hr_anomaly']) | (df['sleep_anomaly']) | (df['steps_anomaly'])]

    st.download_button(
        "Download CSV Report",
        report.to_csv(index=False),
        "anomaly_report.csv"
    )


Overwriting app.py


In [11]:
from pyngrok import ngrok
ngrok.set_auth_token("380ZzaoDdXGvfjMya7VVYPbpJA4_3dcMEvWNppZQDdUGs9j7p")


In [12]:
!streamlit run app.py &>/content/logs.txt &


In [13]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://unspited-olive-inartistic.ngrok-free.dev" -> "http://localhost:8501">